In [1]:
import pandas as pd

import csv
from itertools import islice

def get_row_col(csv_filename, row, col):
    with open(csv_filename, 'rt') as f:
        return next(islice(csv.reader(f), row, row+1))[col]

def import_flow_data(csv_path):
    print(get_row_col(csv_path,5,1))
    flow_raw = pd.read_csv(csv_path, header=10)
    flow_object = DataGroup(get_row_col(csv_path,5,1), get_row_col(csv_path,5,0), flow_raw)
    return flow_object

# flow_raw = import_flow_data('../Data/Spillimacheen/08NA011_QR_Apr-2-2020_06_01_41PM.csv')
    

In [2]:
class DataGroup:
    def __init__(self, river_gauge_title, river_gauge_id, df_river_flow):
        self.river_gauge_title = river_gauge_title
        self.river_gauge_id = river_gauge_id
        self.df = df_river_flow
        print(self.df.info)
        print(f"Created ")
    
        


In [3]:
# flow_raw = import_flow_data('../Data/Spillimacheen/08NA011_QR_Apr-2-2020_06_01_41PM.csv')
flow_csv_path = '../Data/Spillimacheen/smaller_river_test_set.csv'

flow_df_raw = pd.read_csv(flow_csv_path, header=7, index_col=0)
flow_df_raw.drop(["Parameter "],axis=1,inplace=True)
flow_df_raw.index = pd.to_datetime(flow_df_raw.index)

flow = pd.DataFrame()
flow['flow {m3/s)'] = flow_df_raw['Value (m3/s)'].resample('H').first()
print(get_row_col(flow_csv_path,5,1))


SPILLIMACHEEN RIVER NEAR SPILLIMACHEEN


In [4]:
# snow_pillow_height_path = '../Data/Spillimacheen/snow-pillow-conrad-glacier/DataSetExport-1585850817180.csv'
# df_snow = pd.read_csv(snow_pillow_height_path, header=2)



In [13]:
import os
import pytz

# MTC = pytz.timezone('MTC')
df_snow = pd.DataFrame()
folder_to_loop = "../Data/Spillimacheen/snow-pillow-conrad-glacier"
for root, dirs, files in os.walk(folder_to_loop):
    for filename in files:
        temp=pd.read_csv(folder_to_loop+'/'+filename, header=2, index_col=0)
        temp.index = pd.to_datetime(temp.index)
        temp = temp.iloc[:,0:1]
        print(temp.columns[0])
        if len(df_snow) == 0:
            df_snow = temp
        df_snow = pd.concat([df_snow,temp], axis=1)
        # pd.merge(temp, df_snow, left_index=True)
        
        # print(filename)
df_snow = df_snow[['Value (Celsius)', 'Value (Centimetres)']].dropna()
print(df_snow.head(10))

Value (Centimetres)
Value (Celsius)
                     Value (Celsius)  Value (Centimetres)  Value (Centimetres)
Timestamp (UTC)                                                               
2018-11-01 15:00:00             -6.2                156.3                156.3
2018-11-01 16:00:00             -6.2                156.3                156.3
2018-11-01 17:00:00             -5.9                156.7                156.7
2018-11-01 18:00:00             -5.4                156.5                156.5
2018-11-01 19:00:00             -4.2                156.6                156.6
2018-11-01 20:00:00             -5.0                156.4                156.4
2018-11-01 21:00:00             -4.6                156.4                156.4
2018-11-01 22:00:00             -4.6                156.2                156.2
2018-11-01 23:00:00             -4.7                156.2                156.2
2018-11-02 00:00:00             -4.6                156.1                156.1
